In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Medical_Chatbot/ai-medical-chatbot.csv')

In [ ]:
data.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


In [ ]:
data['Doctor'].value_counts()

,count
Doctor,
Hi. For further doubts consult a sexologist online -->,1519
"Hi. How are you doing? Yes, as you have heard, Yasmin can be used as an emergency pill. Even if you have not ejaculated into her, she stands a chance of pregnancy if the pre-seminal fluid, the clear fluid that comes out before semen, which is rich in young healthy sperms, comes in contact with her genitals. As soon as possible, earlier the better, I suggest taking four tablets of Yasmin (combination of Ethinyl Estradiol and Drospirenone) at the time, repeat four more tablets after 12 hours. This being a high dose of hormones, it will suddenly increase the inner lining of the uterus. Then as it is a single dose, the inner lining will fall off. This makes it unfavorable for the possible embryo or baby to settle into the uterus. She will get her periods within 7 to 10 days of the last pill. But do not keep repeating this frequently, it can disturb the hormonal balance and can reduce the effectiveness of the pill. Consult your specialist doctor, discuss with him or her and start taking the medicines after their consent. This is an old method, but it is an effective emergency method. Just do not delay the pills. As time passes by, the effectiveness reduces. Hope I have clarified your query, do write back. All the best. For further queries consult an obstetrician and gynaecologist online -->",1137
"Hello. For PCOD (polycystic ovarian disease), powder Kalachikai has a good effect, but the duration of treatment is long (more than three months). For proper evaluation of disease, please go for USG whole abdomen and other investigations. If the size and number of cysts are more, then I suggest consulting a gynecologist as you may require surgery. If the number and size of cysts are less, you may continue treatment along with syrup Evecare 10 ml BD and Kanchnar Guggulu two BD. For more information consult an ayurveda specialist online -->",1137
"Hello. As the serum beta hCG levels are less than 3 mIU/mL, the pregnancy is definitely ruled out. The delay in the menses is mostly due to sex induced stress after the first time when reproductive hormones readjust themselves. The menses can get delayed even up to two weeks, so just wait for another week and they should arrive. If not, then you can safely opt for a progesterone withdrawal to resume menses. For further information consult an obstetrician and gynaecologist online -->",1137
"Hello, Wellcome to iclinq.com. There are few researches/studies on cocaine use by males and its effect on pregnancy. Few suggest that cocaine by itself has limited effects as most of the time it is taken along with other drugs or as a cocktail of alcohol and cigarette (tobacco). So, most of the people take not just cocaine but a combination of drugs. Cocaine narrows blood vessels (vasoconstriction). It can lead to erectile dysfunction. Few studies suggest that it has receptors on testicles and sperm. So, it can degenerate testicular tissues/or sperm quality, transfer from sperm to female egg and can lead to early miscarriage. Cocaine is a very fast acting drug which affects the nervous system and produces short-lived euphoric attitude for 15 minutes to an hour, but causes long-term damage to the body and brain like anxiety, depression, aggression, impairment of logic and critical thinking, heart problem, hypertension and decrease in bone density. Its half-life is an hour. So, it takes about an hour for half of the cocaine consumed to leave the body. But, with long-term use, the drug starts to accumulate in the blood and body tissues allowing certain tests to detect it in the system for an extended period of time. After a single use of cocaine, agents created by its metabolism can be detected in the urine for two to four days, and in chronic users, cocaine can be detected up to 12 days and highly concentrated cocaine can be detected in the urine up to three weeks. It can be detected in the saliva and blood for an average 12-48 hours. In h

In [ ]:
data.shape

(256916, 3)

In [ ]:
data['Question']=data['Description']+data['Patient']
data['Answer']=data['Doctor']

In [ ]:
data[:1000]['Patient']

,Patient
0,"Hi doctor,I am just wondering what is abutting..."
1,"Hi doctor, I am a 22-year-old female who was d..."
2,Hi doctor! I used to have clear skin but since...
3,"Hello doctor,I am having an uncomfortable feel..."
4,"Hello doctor,Before two years had sex with a c..."
...,...
995,"Hello doctor, My lax les is 38 cm with inflame..."
996,"Hello doctor,I want to get some information re..."
997,"Hello doctor, I went to the chiropractor and g..."
998,"Hello doctor,I am concerned about a possible r..."


In [ ]:
chat= data[:1000].copy()

In [ ]:
def clean_text(text):
    # Remove special characters and numbers, but keep essential punctuation
    text = re.sub(r'[^a-zA-Z?.!,\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing to the questions and answers
chat['processed_question'] = chat['Question'].apply(clean_text)
chat['processed_answer'] = chat['Answer'].apply(clean_text)

In [ ]:
chat['processed_question'].isnull().sum()

np.int64(0)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
question_embeddings = model.encode(chat['processed_question'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
def get_response(user_input):
    user_input = clean_text(user_input)

    user_embedding = model.encode([user_input])

    similarities = cosine_similarity(user_embedding, question_embeddings).flatten()

    closest_match_index = np.argmax(similarities)

    response = chat['processed_answer'].iloc[closest_match_index]
    return response

user_input = "What does abutment of the nerve root mean?"
response = get_response(user_input)
print(f"Doctor's Response: {response}")

Doctor's Response: hi. i have gone through your query with diligence and would like you to know that i am here to help you. for further information consult a neurologist online


In [ ]:
def chat_with_me():
    print("Chatbot: Hello, how can I help you today?")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = get_response(user_input)
        print("Chatbot: ", response)


chat_with_me()

Chatbot: Hello, how can I help you today?
You: Kindly explain about Beta Strep.
Chatbot:  hello. there are lots of bacteria and other organisms that colonize healthy skin. yeast a fungus also is a commensal in our body. just the presence of these organisms is of no significance. the colony count is needed, which is high, will need a course of antibiotics, which can be decided by culture and sensitivity. but after a surgery like what you had, or any significant stress or use of lots of antibiotics as would have happened after surgery or use of steroids or diabetes, all infections show an altered balance between good and bad bacteria. you do not need any treatment for this. just wait for your body to develop those good bacteria, and gradually, all tests will come negative. if you have any symptoms, treat as per need. avoid unnecessary antibiotics use. they will further delay the recovery. i hope i have clarified your query, do write back if any more questions.
You: Q. I am suffering from

In [ ]:
import os
import joblib

In [ ]:
# Set up paths
model_save_path = '/content/drive/MyDrive/Medical_Chatbot/saved_model'
embeddings_save_path = '/content/drive/MyDrive/Medical_Chatbot/question_embeddings.pkl'
chat_csv_path = '/content/drive/MyDrive/Medical_Chatbot/chat_data.csv'

# Create the directory if it doesn't exist
os.makedirs(model_save_path, exist_ok=True)

# Save SentenceTransformer model
model.save(model_save_path)

# Save embeddings and chat data
joblib.dump(question_embeddings, embeddings_save_path)
chat.to_csv(chat_csv_path, index=False)